# Workshop 1 - Apply
    
In this notebook you will work on a homework assignment involving a Vierendeel frame.

Our matrix method implementation is now completely stored in a local package, consisting of three classes.

## Vierendeel frame

<center><figure>
  <IMG SRC="https://raw.githubusercontent.com/ibcmrocha/public/main/vierendeel.png" WIDTH=400 ALIGN="center">
      </figure></center>

With:

- $h = 1$
- $b = 1$
- $EI_r = 10000$
- $EI_k = 1000$
- $EA  = 1\cdot 10^{10}$
- $H = 100$


In the first half of this course last quarter, you have learned that the deformation of Vierendeel frames (an example of which is shown above) can be obtained in a simplified way by assuming the global deformation can be described by a shear beam with equivalent stiffness given by:

$$
k = \frac{24}{h\left(\displaystyle\frac{h}{EI_k}+\frac{b}{EI_r}\right)}
$$

Now that you have the tools to solve the original frame problem using the Matrix Method, your task in this assignment is to investigate the validity of this equivalent shear beam model.

Note that the checks only had a single element. For this model you need to obtain $\mathbf{K}$ and $\mathbf{f}$ of all elements and add them to the correct locations in the global stiffness matrix and force vector. To do that, make use of the `global_dofs` function of the Element class and the `np.ix_` Numpy utility function. (Tip: refer back to what you did in the `constrain` function).

Once you have a solution, use SymPy / Maple / pen and paper to solve a shear beam problem with the equivalent stiffness given above (It is very similar to the simple extension problem above) and compare the horizontal displacement at the point of application of $H$ for the two models.

Investigate how the two models compare for different values of $EA$, ranging from very small (*e.g.* $1\cdot 10^{-5}$) to very large (*e.g.* $1\cdot10^{10}$). What explains the behavior you observe?

In [1]:
import numpy as np
import matrixmethod as mm
%config InlineBackend.figure_formats = ['svg']

In [2]:
mm.Node.clear()
mm.Element.clear()

In [3]:
h = 1
b = 1
EI_r = 10000
EI_k = 1000
EA = 1 * 10**10
H = 100

node = []
node.append(mm.Node(0,0))
node.append(mm.Node(b,0))
node.append(mm.Node(b,h))
node.append(mm.Node(0,h))

elements = []
elements.append(mm.Element(node[0], node[1]))
elements.append(mm.Element(node[1], node[2]))
elements.append(mm.Element(node[2], node[3]))
elements.append(mm.Element(node[0], node[3]))

Horizontal = {}
Horizontal['EI'] = EI_r
Horizontal['EA'] = EA
elements[0].set_section(Horizontal)
elements[2].set_section(Horizontal)

Vertical = {}
Vertical['EI'] = EI_k
Vertical['EA'] = EA
elements[1].set_section(Vertical)
elements[3].set_section(Vertical)

constraints = mm.Constrainer()

constraints.fix_dof(node[0], 0)
constraints.fix_dof(node[0], 1)
constraints.fix_dof(node[1], 1)

node[3].add_load([H,0,0])

for elem in elements:
    print(elem)

Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=1,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]
Element connecting:
node #1:
 This node has:
 - x coordinate=1,
 - z coordinate=0,
 - degrees of freedom=[3, 4, 5],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=1,
 - z coordinate=1,
 - degrees of freedom=[6, 7, 8],
 - load vector=[0. 0. 0.]
Element connecting:
node #1:
 This node has:
 - x coordinate=1,
 - z coordinate=1,
 - degrees of freedom=[6, 7, 8],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordinate=0,
 - z coordinate=1,
 - degrees of freedom=[9, 10, 11],
 - load vector=[100.   0.   0.]
Element connecting:
node #1:
 This node has:
 - x coordinate=0,
 - z coordinate=0,
 - degrees of freedom=[0, 1, 2],
 - load vector=[0. 0. 0.]
with node #2:
 This node has:
 - x coordina

In [4]:
global_k = np.zeros((3*len(node), 3*len(node)))
global_f = np.zeros(3*len(node))

for elem in elements:
    elmat = elem.stiffness()
    idofs = elem.global_dofs()
    
    global_k[np.ix_(idofs,idofs)] += elmat

for n in node:
    global_f[n.dofs] += n.p

In [5]:
Kff, Ff = constraints.constrain ( global_k, global_f )
u = np.matmul ( np.linalg.inv(Kff), Ff )
print(u)

[ 4.16667982e-04  4.99999539e-09  4.16665674e-04  4.58333833e-03
 -4.99999807e-09  4.16675352e-04  4.58334333e-03  4.99999806e-09
  4.16677659e-04]


In [6]:
#provided in case you want to solve the shear beam problem using SymPy
import sympy as sym
x, k, L, H = sym.symbols('x, k, L, H')
w = sym.Function('w')

ODE_shear = sym.Eq(w(x).diff(x, 2)*k, 0)
w = sym.dsolve(ODE_shear, w(x)).rhs

gamma = w.diff(x)
V = k*gamma
eq1 = sym.Eq(w.subs(x,0), 0)
eq2 = sym.Eq(V.subs(x,L), H)
Constants = sym.solve([eq1, eq2], sym.symbols('C1, C2'))
display(w.subs(Constants))



H*x/k